# This is a minimal example for how to use the Complex inversion

In [1]:
import complex_inversion_tools as cit
import numpy as np
import matplotlib.pyplot as plt

First, we create the synthetic complex measurements.
* We define a forward operator (np.log)
* We define a method to calculate the Jacobian
* We create synthetic measurements from a true model ```m_true```. The synthetic measurements must be saved into an array that fullfills the Conjugate Coordinate form.

In [2]:
# create forward operator

def f(m):
    """
    This is the forward operator used to create the synthetic measurements and during the inversion.
    Please excuse the inverse crime in the spirit of keeping things simple.
    """
    return np.log(m)

def J(m):
    """
    This method calculates the complex jacobian. Please note that the forward operator is analytic.
    """
    return np.diagflat(1 / m)
    

# create synthetic measurements
M = 10 # number of model parameters

m_true = np.linspace(1, 10, M) + np.linspace(0.1, 1, M) * 1j # Here we define the true complex model
d = f(m_true) # here we calculate the synthetic data to be inverted

d = np.hstack((d, d.conj())) # here we convert the data into the conjugate coordinate form

To perform the inversion we have to set up a couple of things.
* First, we define a function ```solve_forward_problem```. This function takes as input a complex valued model in the conjugate coordinate form. It must return the associated forward response in conjugate coordinate form, as well as the Jacobian (by default not in CC form, optional in CC form)
* Second we define a starting model for the inversion in CC form.
* Third we define the inverse data covariance matrix in CC form as explained in the Paper.
* Fourth we define the regularization operator in CC form.

In [3]:
# Inversion preparation

def solve_forward_problem(m):
    fm = f(m)
    return fm, J(m[:M])

m0 = np.ones(M) + 1j * np.ones(M)
m0 = np.hstack((m0, m0.conj()))

Rd = np.eye(2 * M)
Rm = np.zeros((2 * M, 2 * M)) # since we have an equally determined problem, we do not need regularization

Finally, we initialize the complex inversion Manager and provide all the necessary input defined above.
After that we are ready to run the inversion.

In [4]:
ciManager = cit.Complex_Inversion_Manager(
    m=m0, # starting model in CC form
    d=d, # data in CC form
    solve_forward_problem=solve_forward_problem, # function returning the forward response and the Jacobain
    lam=0, # scalar scaling the complex regularization term.
    Rd=Rd, # inverse data covariance matrix in CC form
    Rm=Rm, # regularization operator in CC form
    mp=np.zeros_like(m0)) # prior model in CC form

ciManager.inversion() # run the inversion

1.5231579425800041 1.3600680703247086 0.6857295109062862
rmse  (1.5231579425800041, 1.3600680703247086, 0.6857295109062862)
eta  1.0391300943921382
rmse  (0.6315107774942494, 0.5106146391190634, 0.37158922536680217)
eta  1.0566731704118772
rmse  (0.15458575327881138, 0.08201883318153283, 0.13103307262030237)
eta  1.021462930921831
rmse  (0.013845976650507298, 0.006529326500585117, 0.012209789713797294)
eta  0.9977462731056235
rmse  (0.00015234051742296496, 0.00013713484952699952, 6.634505478107536e-05)
eta  0.9999252221144684
rmse  (1.6529652262351414e-08, 1.204789544905181e-08, 1.1317138293887545e-08)
eta  1.0000000163127563


Checking the inversion result against the true model used to generate the synthetic data shows that our inversion was successfull! Nice!

In [5]:
np.allclose(ciManager.m[:M], m_true)

True